# BQL Object Model Basics
You can use Bloomberg Query Language (BQL) via the PyBQL API to request Bloomberg data from within BQuant notebooks. There are two main ways to use BQL: the object model and the string interface. This notebook provides an introduction to the object model. In this notebook, we'll start with a simple example and build on it to learn how to construct more complex requests.

<a href = "#A">Example A: Requesting a Single Data Item</a>  
<a href = "#B">Example B: Requesting Multiple Data Items</a>  
<a href = "#C">Example C: Requesting Time Series and Point-In-Time Data</a>  
<a href = "#D">Example D: Requesting Data for Multiple Securities</a>  
<a href = "#related"> Related Resources</a>  
<br>

## Example A: Requesting a Single Data Item <a name="A"></a>
In this example, we'll create and execute the simplest possible request: a single data item for a single security. We'll build on this request in later examples. For now, let's request the last price for AAPL US Equity. 
We'll use three main BQL functions:

- **bq.data**: Defines the data item to include in the request.  
        `bq.data. + field(arguments)`
- **bql.Request()**: Generates the request.  
        `bql.Request(security or universe, data items)`
- **bq.execute()**: Executes the request.
        `bq.execute(request)`
        
_Note: For more information on finding data fields that can be queried with BQL, see <a href = "#related"> Related Resources</a> ._

### Example Code

In [1]:
# Import the BQL library
import bql

# Instantiate an object to interface with the BQL service
bq = bql.Service()

# Define a variable for the security 
security = "AAPL US Equity"

# Define a data item for the last price field
# In this example, we're not passing any arguments
last = bq.data.px_last()

# Generate the request using the sercurity variable and data item
request =  bql.Request(security, last)

# Execute the request
response = bq.execute(request)

# Display the response in a DataFrame
response[0].df()

,DATE,CURRENCY,PX_LAST()
ID,,,
AAPL US Equity,2020-04-06,USD,253.259995


<div style="text-align: right"><a href="#BQL-Object-Model-Basics">Return to Top ↑</a></div>

## Example B: Requesting Multiple Data Items <a name="B"></a>
Now that we've seen how to request a single data item, let's expand the previous example to request additional data items. With this example, we'll see how the object model allows us to build up requests in parts. We'll again use AAPL US Equity, but this time we'll request the last price, high price, and low price.

### Example Code

In [2]:
# Import the BQL library
import bql

# Instantiate an object to interface with the BQL service
bq = bql.Service()

# Define a variable for the security 
security = "AAPL US Equity"

# Define data items for the pricing fields
last = bq.data.px_last()
high = bq.data.px_high()
low = bq.data.px_low()

# Generate the request using the security and a list of the data items
request = bql.Request("AAPL US Equity", [last, high, low])

# Execute the request
response = bq.execute(request)

# Display the response in a DataFrame
# Use the combined_df function to display the three returned values in a single DataFrame 
bql.combined_df(response)

,DATE,CURRENCY,PX_LAST(),PX_HIGH(),PX_LOW()
ID,,,,,
AAPL US Equity,2020-04-06,USD,253.309998,254.080002,249.380005


<div style="text-align: right"><a href="#BQL-Object-Model-Basics">Return to Top ↑</a></div>

## Example C: Requesting Time Series and Point-In-Time Data <a name="C"></a>
In the examples above, we didn't specify a data date so BQL returned data for today's date by default. If we want to return data for a specific date or date range, we can pass start and end values as arguments to the BQL range function. Note that the dates are associated with the data items, not the request.

_Note: To request point-in-time data (for a single date), pass the same value for the start and end parameters._


### Example Code

In [3]:
# Import the BQL library
import bql

# Instantiate an object to interface with the BQL service
bq = bql.Service()

# Define the date range for the request
date_range = bq.func.range('2017-06-05','2017-06-09')

# Define data items for the pricing fields
# Pass the defined date range
last = bq.data.px_last(dates=date_range)
high = bq.data.px_high(dates=date_range)
low = bq.data.px_low(dates=date_range)

# Generate the request using the security ticker and a list of the data items
request = bql.Request("AAPL US Equity", [last, high, low])

# Execute the request
response = bq.execute(request)

# Display the response in a data frame
# Use the combined_df function to display 
# the three returned values in a single data frame 
bql.combined_df(response)

,DATE,CURRENCY,"PX_LAST(dates=RANGE(2017-06-05,2017-06-09))","PX_HIGH(dates=RANGE(2017-06-05,2017-06-09))","PX_LOW(dates=RANGE(2017-06-05,2017-06-09))"
ID,,,,,
AAPL US Equity,2017-06-05,USD,153.93,154.45,153.46
AAPL US Equity,2017-06-06,USD,154.45,155.81,153.78
AAPL US Equity,2017-06-07,USD,155.37,155.98,154.48
AAPL US Equity,2017-06-08,USD,154.99,155.54,154.40
AAPL US Equity,2017-06-09,USD,148.98,155.19,146.02


<div style="text-align: right"><a href="#BQL-Object-Model-Basics">Return to Top ↑</a></div>

## Example D: Requesting Data for Multiple Securities <a name="D"></a>
Now that we know how to pull multiple data items across a range of dates for a single security, let's look at a few different ways to define a universe of securities:

- Define a list of securities. For example `securities = ['MSFT US Equity', 'IBM US Equity', 'AMZN US Equity']`
- Call a BQL function that returns a list of securities, such as the `bq.univ.members()` function, which returns the members of an index.

In this example, we'll use bq.univ.members() to pass the members of the INDU Index.

### Example Code

In [4]:
# Import the BQL library
import bql

# Instantiate an object to interface with the BQL service
bq = bql.Service()

# Define a universe of the members of the INDU Index
indu = bq.univ.members("INDU Index")

# Define the date range for the request
date_range = bq.func.range('2017-06-05','2017-06-09')

# Define data items for the pricing fields
# Pass the defined date range
last = bq.data.px_last(dates=date_range)
high = bq.data.px_high(dates=date_range)
low = bq.data.px_low(dates=date_range)

# Generate the request using the security universe and a list of the data items
request = bql.Request(indu, [last, high, low])

# Execute the request
response = bq.execute(request)

# Display the response in a DataFrame
# Use the combined_df function to display 
# the three returned values in a single DataFrame
# To verify the output, use tail(3) to show the last three rows of the response
bql.combined_df(response).tail(3)

,DATE,CURRENCY,"PX_LAST(dates=RANGE(2017-06-05,2017-06-09))","PX_HIGH(dates=RANGE(2017-06-05,2017-06-09))","PX_LOW(dates=RANGE(2017-06-05,2017-06-09))"
ID,,,,,
NKE UN Equity,2017-06-07,USD,53.23,53.36,52.75
NKE UN Equity,2017-06-08,USD,53.20,54.14,53.15
NKE UN Equity,2017-06-09,USD,53.46,53.47,52.81


<div style="text-align: right"><a href="#BQL-Object-Model-Basics">Return to Top ↑</a></div>

## Related Resources <a name="related"></a>
- <a href = "./BQL String Interface Basics.ipynb">BQL String Interface Basics</a>
- <a href = "exampleroot/53815e8850a54b4187fb0d1335ee39d5/BQL Field Function and Universe Search.ipynb">BQL Field, Function, and Universe Search</a>
- <a href = "exampleroot/9032931110d6467fa8f1938424453f06/_BQL Quick Reference Guide.ipynb">BQL Quick Reference Guide</a>
- <a href = "exampleroot/f2d2cebf9dac40699094b3caa8ad35ce/BQL Fundamental Data.ipynb">BQL Fundamental Data</a>
